In [32]:
import requests
from bs4 import BeautifulSoup

# FYIs (still need fixing): 

### name change for diff case IDs (older include UKSC)
### name change for older citation #s (Neutral Citation v Neutral Citation Number v Neutral Citation Number(s)
### recent cases with multipar issues/summaries

### FUNCTION todos: 

define each function & tweak to iterate over many pages (list of each dict output)
combine into single script
write main with elifs for NOT exceptions first 
concatenate each dictionary into single dataframe (two dataframes? one for more detail and one for larger 

write tests??? 

# 29 April 2020 - present

this works for all* 2022 cases

*MOST -- some 2022 cases have the same problem as below (Akdogan, PWR, Bloomberg)

for 2020/2021 cases, the problem is just multiple p tags for the issue and fact h4 tags--need to address this so the dictionary doesn't get messed up. should work otherwise

In [54]:
response = requests.get('https://www.supremecourt.uk/cases/uksc-2020-0195.html')
soup = BeautifulSoup(response.content, 'html.parser')

In [55]:
case_details = []
for detail in soup.find_all('div', class_='panel-filled'): 
    case_name = detail.find('span').string
    case_id = detail.find('h3', class_='sc-access').string
    case_id = case_id.replace('Case ID: ', '')
    keys = []
    for d in detail.find_all('h4'):
        key = d.string
        keys.append(key)
    values = []
    for d in detail.find_all('p'):
        value = d.string
        values.append(value)
    case_details.append({'Keys': keys, 'Values': values})
    case_details = case_details[0]

case_dict = {case_details['Keys'][i]: case_details['Values'][i] for i in range(len(case_details['Keys']))}
case_dict['Name'] = case_name
case_dict['Case ID'] = case_id
case_dict

{'Issue': 'Whether the UK Government has recognised Interim President Guaido as Head of State of Venezuela and, if so, whether any challenge to the validity of Mr Guaido’s appointments to the Board of the Central Bank of Venezuela is justiciable in an English court.',
 'Facts': None,
 'Judgment appealed': '[2020] EWCA Civ 1249',
 'Appellant(s)': '"Guaidó Board" of the Central Bank of Venezuela',
 'Respondent(s)': ' "Maduro Board" of the Central Bank of Venezuela',
 'Justices': 'Lord Reed, Lord Hodge, Lord Lloyd-Jones, Lord Hamblen, Lord Leggatt',
 'Hearing start date': '19 July 2021',
 'Hearing finish date': '22 July 2021 ',
 'Judgment date': '20 December 2021',
 'Neutral citation': '[2021] UKSC 57',
 'Name': 'Maduro Board of the Central Bank of Venezuela (Respondent/Cross-Appellant) v Guaidó Board" of the Central Bank of Venezuela (Appellant/Cross-Respondent)',
 'Case ID': '2020/0195'}

# 13 March 2019 - April 1 2020

this works for ALL cases 13 March 2019 - April 1 2020


WORKS FOR "R v Chaytor and others (Appellants)" 01 DEC 2010
WORKS FOR "Greater Glasgow Health Board..." 17 DEC 2014

In [52]:
response = requests.get('https://www.supremecourt.uk/cases/uksc-2013-0124.html')
soup = BeautifulSoup(response.content, 'html.parser')

In [53]:
case_details = []
for detail in soup.find_all('div', class_='panel-filled'):
    case_name = detail.find('span').string
    keys = []
    for d in detail.find_all('h3', class_='sc-access'):
        key = d.string
        keys.append(key)
    values = []
    for d in detail.find_all('p'):
        value = d.string
        values.append(value)
    case_details.append({'Keys': keys, 'Values': values})
    case_details = case_details[0]

case_dict = {case_details['Keys'][i]: case_details['Values'][i] for i in range(4)}
case_dict['Name'] = case_name
case_dict

{'Judgment date': '17 Dec 2014',
 'Neutral citation number': '[2014] UKSC 68',
 'Case ID': 'UKSC 2013/0124',
 'Justices': 'Lady Hale, Lord Wilson, Lord Reed, Lord Hughes, Lord Hodge',
 'Name': 'Greater Glasgow Health Board (Appellant) v Doogan and another (Respondents) (Scotland)'}

# 29 October 2009 - 6 March 2019

works from October 2009 (first case) - 6 March 2019

EXCEPT DEC 2014 -- Loveridge Appellant, Greater Glasgow  **New function for this one defined below**

EXCEPT DEC 2012 -- Spiller and other, R v Chaytor **New function for this one defined below**

In [57]:
response = requests.get('https://www.supremecourt.uk/cases/uksc-2009-0180.html')
soup = BeautifulSoup(response.content, 'html.parser')

In [58]:
case_details = []
for detail in soup.find_all('div', class_='panel-filled'):
    case_name = detail.find('span').string
    keys = []
    for d in detail.find_all('h5'):
        key = d.string
        keys.append(key)
    values = []
    for d in detail.find_all('p'):
        value = d.string
        values.append(value)
    case_details.append({'Name': case_name, 'Keys': keys, 'Values': values})
    case_details = case_details[0]

case_dict = {case_details['Keys'][i]: case_details['Values'][i] for i in range(4)}
case_dict['Name'] = case_details['Name']
case_dict

{'Judgment date': '09 Feb 2011 & 03 Nov 2010',
 'Neutral citation number': '[2011] UKSC 6 & [2010] UKSC 45',
 'Case ID': 'UKSC 2009/0180',
 'Justices': 'Lord Phillips, Lord Hope, Lord Rodger, Lord Walker, Lady Hale, Lord Brown, Lord Mance, Lord Neuberger, Lord Collins',
 'Name': 'Manchester City Council (Respondent) v Pinnock (Appellant)'}

# January 2010

cases with MORE THAN two citation numbers (first three Jan 2010) 

In [59]:
response = requests.get('https://www.supremecourt.uk/cases/uksc-2009-0016.html')
soup = BeautifulSoup(response.content, 'html.parser')

In [60]:
case_details = []
for detail in soup.find_all('div', class_='panel-filled'):
    case_name = detail.find('span').string
    keys = []
    for d in detail.find_all('h5'):
        key = d.string
        keys.append(key)
    keys.remove('Neutral citation number(s)')
    values = []
    for d in detail.find_all('p'):
        value = d.string
        values.append(value)
    citations = []
    for d in detail.find('ul'):
        cit = d.string
        citations.append(cit)
    citations = [citation for citation in citations if citation != '\n']
    citations = ' & '.join(citations)
    case_details.append({'Keys': keys, 'Values': values})
    case_details = case_details[0]
    
case_dict = {test_details['Keys'][i]: test_details['Values'][i] for i in range(3)}
case_dict['Neutral citation number(s)'] = citations
case_dict['Name'] = case_name
case_dict

{'Judgment date': '27 Jan 2010',
 'Case ID': 'UKSC 2009/0016',
 'Justices': 'Lord Phillips, Lord Hope, Lord Rodger, Lord Walker, Lady Hale, Lord Brown, Lord Mance',
 'Neutral citation number(s)': '[2010] UKSC 1 & [2010] UKSC 2 & [2010] UKSC 5',
 'Name': "Her Majesty's Treasury (Respondents) v Mohammed Jabar Ahmed and others (FC) (Appellants) "}

# 3 December 2014

Loveridge (Appellant)

In [61]:
response = requests.get('https://www.supremecourt.uk/cases/uksc-2013-0273.html')
soup = BeautifulSoup(response.content, 'html.parser')

In [62]:
case_details = []
for detail in soup.find_all('div', class_='panel-filled'):
    case_name = detail.find('span').string
    keys = ['Judgment date']
    for d in detail.find_all('h4'):
        key = d.string
        keys.append(key)
    values = []
    for d in detail.find_all('p'):
        value = d.string
        values.append(value)
    case_details.append({'Name': case_name, 'Keys': keys, 'Values': values})
    case_details = case_details[0]

case_dict = {case_details['Keys'][i]: case_details['Values'][i] for i in range(len(case_details['Values']))}
case_dict['Name'] = case_details['Name']
case_dict

{'Judgment date': '03 Dec 2014',
 'Neutral citation number': '[2014] UKSC 65',
 'Case ID': 'UKSC 2013/0273',
 'Justices': 'Lord Neuberger, Lord Wilson, Lord Sumption, Lord Carnwath, Lord Toulson',
 'Name': 'Loveridge (Appellant) v Mayor and Burgesses of the London Borough of Lambeth (Respondent) '}

# 01 December 2010

Spiller and another (Appellant)

In [63]:
response = requests.get('https://www.supremecourt.uk/cases/uksc-2009-0210.html')
soup = BeautifulSoup(response.content, 'html.parser')

In [64]:
case_details = []
for detail in soup.find_all('div', class_='panel-filled'):
    case_name = detail.find('h3').string
    keys = []
    for d in detail.find_all('h5'):
        key = d.string
        keys.append(key)
    values = []
    for d in detail.find_all('p'):
        value = d.string
        values.append(value)
    case_details.append({'Name': case_name, 'Keys': keys, 'Values': values})
    case_details = case_details[0]

case_dict = {case_details['Keys'][i]: case_details['Values'][i] for i in range(4)}
case_dict['Name'] = case_details['Name']
case_dict

{'Judgment date': '01 Dec 2010',
 'Neutral citation number': '[2010] UKSC 53',
 'Case ID': 'UKSC 2009/0210',
 'Justices': 'Lord Phillips, Lord Rodger, Lord Walker, Lord Brown, Sir John Dyson',
 'Name': 'Spiller and another (Appellants) v Joseph and others (Respondents)'}